SVM Linear Kernal 

In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random


# Importing the dataset
dataset = pd.read_csv('/Users/fabianreginold/Downloads/traice/traice_moneylaundering/PS_20174392719_1491204439457_log.csv')
dataset.drop('nameOrig', axis=1, inplace=True)
dataset.drop('nameDest', axis=1, inplace=True)
dataset.drop('isFlaggedFraud', axis=1, inplace=True)

In [10]:
dict(dataset)

{'step': 0            1
 1            1
 2            1
 3            1
 4            1
           ... 
 6362615    743
 6362616    743
 6362617    743
 6362618    743
 6362619    743
 Name: step, Length: 6362620, dtype: int64,
 'type': 0           PAYMENT
 1           PAYMENT
 2          TRANSFER
 3          CASH_OUT
 4           PAYMENT
              ...   
 6362615    CASH_OUT
 6362616    TRANSFER
 6362617    CASH_OUT
 6362618    TRANSFER
 6362619    CASH_OUT
 Name: type, Length: 6362620, dtype: object,
 'amount': 0             9839.64
 1             1864.28
 2              181.00
 3              181.00
 4            11668.14
               ...    
 6362615     339682.13
 6362616    6311409.28
 6362617    6311409.28
 6362618     850002.52
 6362619     850002.52
 Name: amount, Length: 6362620, dtype: float64,
 'oldbalanceOrg': 0           170136.00
 1            21249.00
 2              181.00
 3              181.00
 4            41554.00
               ...    
 6362615     339682.13

In [11]:

dataset.type.unique()

array(['PAYMENT', 'TRANSFER', 'CASH_OUT', 'DEBIT', 'CASH_IN'],
      dtype=object)

In [12]:
sample_dataframe = dataset.sample(random_state=42,n=100000)
sample_dataframe[sample_dataframe['isFraud']==1]

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
6303116,683,CASH_OUT,93483.23,93483.23,0.00,86882.30,180365.53,1
3960303,294,CASH_OUT,806863.30,806863.30,0.00,2344186.16,3151049.45,1
2736446,212,TRANSFER,4953893.08,4953893.08,4953893.08,0.00,0.00,1
1030272,51,CASH_OUT,1170282.92,1170282.92,0.00,173264.14,1343547.06,1
1212611,133,CASH_OUT,1159010.44,1159010.44,0.00,175085.52,1334095.97,1
...,...,...,...,...,...,...,...,...
6122464,540,CASH_OUT,3707342.38,3707342.38,0.00,558618.20,4265960.58,1
6273168,633,TRANSFER,125107.12,125107.12,0.00,0.00,0.00,1
6008702,438,TRANSFER,1648547.69,1648547.69,0.00,0.00,0.00,1
6168680,557,CASH_OUT,3571807.73,3571807.73,0.00,527694.86,4099502.59,1


In [13]:
X=sample_dataframe
unique_types=X['type'].unique()
unique_types

array(['CASH_IN', 'PAYMENT', 'TRANSFER', 'CASH_OUT', 'DEBIT'],
      dtype=object)

In [14]:
#one hot encoding using pd dummies
for each_categorical_value in X['type'].unique():
    X[each_categorical_value] = pd.get_dummies(X['type'])[each_categorical_value]
    
#X['CASH_IN'] = pd.get_dummies(X['type'])['CASH_IN']
#X['CASH_OUT'] = pd.get_dummies(X['type'])['CASH_OUT']
#X['DEBIT'] = pd.get_dummies(X['type'])['DEBIT']
#['PAYMENT'] = pd.get_dummies(X['type'])['PAYMENT']
#X['TRANSFER'] = pd.get_dummies(X['type'])['TRANSFER']
#X.drop('CASH_IN',axis=True,inplace=True)
X.drop('type',axis=True,inplace=True)
X.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,CASH_IN,PAYMENT,TRANSFER,CASH_OUT,DEBIT
3737323,278,330218.42,20866.00,351084.42,452419.57,122201.15,0,1,0,0,0,0
264914,15,11647.08,30370.00,18722.92,0.00,0.00,0,0,1,0,0,0
85647,10,152264.21,106589.00,258853.21,201303.01,49038.80,0,1,0,0,0,0
5899326,403,1551760.63,0.00,0.00,3198359.45,4750120.08,0,0,0,1,0,0
2544263,206,78172.30,2921331.58,2999503.88,415821.90,337649.60,0,1,0,0,0,0


In [15]:
X = sample_dataframe.iloc[:, [0,1,2,3,4,5,7,8,9,10,11]].values
y = sample_dataframe.iloc[:, 6].values

y

array([0, 0, 0, ..., 0, 0, 0])

In [16]:
# Splitting the dataset into the Training set and Test set

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
#X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=1)

counts = np.unique(y_train, return_counts=True)


In [17]:
# Feature Scaling
import pickle
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
pickle.dump(sc, open('./'+'scaler.pkl', 'wb'))
#X_val = sc.transform(X_val)
X_test = sc.transform(X_test)
print(counts)

(array([0, 1]), array([69899,   101]))


Linear Kernel SVM = AUC 0.96

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [19]:
lr = LogisticRegression()
knn = KNeighborsClassifier()
svc = SVC()
nb = GaussianNB()
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier()
bc = BaggingClassifier()
abc = AdaBoostClassifier()
gbc = GradientBoostingClassifier()

In [20]:
len(X_train)

70000

In [21]:
import pickle
def trainFitTest(model,space):
    ##models = [lr,knn,svc,nb,dtc,rfc,bc,abc,gbc]
    score = []
    ##for model in models:
    #model.fit(X_train,Y_train)
    cv_inner = KFold(n_splits=10, shuffle=True, random_state=42)
    search = GridSearchCV(model, space, scoring='accuracy', cv=cv_inner, refit=True)
    #print(type(search))
    result = search.fit(X_train, y_train)#training the model
    # get the best performing model fit on the whole training set
    best_model = result.best_estimator_
    pickle.dump(best_model, open('./'+str(model)+'_best_model.pkl', 'wb'))
    print('training score',best_model.score(X_train,y_train))
    print('prediction score - ',best_model.score(X_test,y_test))
    score.append(best_model.score(X_test,y_test))
    Y_rfc_pred = best_model.predict(X_test)
    print(Y_rfc_pred)
    print(classification_report(y_test,Y_rfc_pred))
    print(confusion_matrix(y_test,Y_rfc_pred))

In [22]:
space = dict()
space['n_jobs']=[1,2,5]
trainFitTest(lr,space)

training score 0.9988714285714285
prediction score -  0.9989666666666667
[0 0 0 ... 0 0 0]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     29960
           1       1.00      0.23      0.37        40

    accuracy                           1.00     30000
   macro avg       1.00      0.61      0.68     30000
weighted avg       1.00      1.00      1.00     30000

[[29960     0]
 [   31     9]]


In [23]:
#trainFitTest(knn)

In [24]:
space=dict()
space['max_leaf_nodes'] = [10, 25, 500]
space['max_features'] = [2, 4, 6]
trainFitTest(dtc,space)

training score 0.9994571428571428
prediction score -  0.9992
[0 0 0 ... 0 0 0]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     29960
           1       0.75      0.60      0.67        40

    accuracy                           1.00     30000
   macro avg       0.87      0.80      0.83     30000
weighted avg       1.00      1.00      1.00     30000

[[29952     8]
 [   16    24]]


In [25]:
#trainFitTest(nb)


In [26]:
space=dict()
space['max_leaf_nodes'] = [10, 25, 500]
space['max_features'] = [2, 4, 6]
trainFitTest(rfc,space)

training score 1.0
prediction score -  0.9996333333333334
[0 0 0 ... 0 0 0]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     29960
           1       1.00      0.72      0.84        40

    accuracy                           1.00     30000
   macro avg       1.00      0.86      0.92     30000
weighted avg       1.00      1.00      1.00     30000

[[29960     0]
 [   11    29]]


In [27]:
#trainFitTest(abc)


In [28]:

space=dict()
space['max_leaf_nodes'] = [10, 25, 500]
space['max_features'] = [2, 4, 6]
space['n_estimators']=[10,20,100]
trainFitTest(gbc,space)

training score 0.9992714285714286
prediction score -  0.9991333333333333
[0 0 0 ... 0 0 0]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     29960
           1       0.77      0.50      0.61        40

    accuracy                           1.00     30000
   macro avg       0.88      0.75      0.80     30000
weighted avg       1.00      1.00      1.00     30000

[[29954     6]
 [   20    20]]


In [29]:
pickle.dump(gbc, open('./'+'gbc.pkl', 'wb'))
   

In [42]:
float('193.1')

193.1

In [59]:
#unit testing
rf=pickle.load(open('RandomForestClassifier()_best_model.pkl','rb'))
values=[1,'PAYMENT',9839.64,703,19304,0,0]
x_unit_test=pd.DataFrame([[1,'PAYMENT',9839.64,703,19304,0,0]],columns=['step', 'type', 'amount', 'oldbalanceOrg', 'newbalanceOrig',
       'oldbalanceDest', 'newbalanceDest'])

In [60]:
values=[1,'PAYMENT',9839.64,703,19304,0,0]

In [61]:
#values=np.array(values[1])
x_unit_test

#for each_categorical_value in X['type'].unique():
#    X[each_categorical_value] = pd.get_dummies(X['type'])[each_categorical_value]

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
0,1,PAYMENT,9839.64,703,19304,0,0


In [62]:
x_unit_test

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
0,1,PAYMENT,9839.64,703,19304,0,0


In [63]:
unique_types=['CASH_IN', 'PAYMENT', 'TRANSFER', 'CASH_OUT', 'DEBIT']

In [64]:
for each_categorical_value in unique_types:
    #x_unit_test[each_categorical_value]=0
    #print(each_categorical_value)
    if(x_unit_test['type'][0]==each_categorical_value):
        x_unit_test[each_categorical_value]=1
    else:
        x_unit_test[each_categorical_value]=0
    #x_unit_test[each_categorical_value]=pd.get_dummies

In [65]:
x_unit_test.columns

Index(['step', 'type', 'amount', 'oldbalanceOrg', 'newbalanceOrig',
       'oldbalanceDest', 'newbalanceDest', 'CASH_IN', 'PAYMENT', 'TRANSFER',
       'CASH_OUT', 'DEBIT'],
      dtype='object')

In [66]:
x_unit_test=x_unit_test[['step', 'type', 'amount', 'oldbalanceOrg', 'newbalanceOrig',
       'oldbalanceDest', 'newbalanceDest','CASH_IN','PAYMENT', 'TRANSFER', 'CASH_OUT',
       'DEBIT']]

In [67]:
#x_unit_test[x_unit_test['type'][0]]=pd.get_dummies(x_unit_test['type'])[x_unit_test['type'][0]]

In [68]:
x_unit_test.drop(columns=['type'],inplace=True)

In [69]:
#x_unit_test.columns=xcopy.columns

In [70]:
sc=pickle.load(open('scaler.pkl','rb'))
x_unit_test_scales=sc.transform(x_unit_test)

In [71]:
rf=pickle.load(open('RandomForestClassifier()_best_model.pkl','rb'))
rf.predict(x_unit_test_scales)

array([1])

In [30]:
import pymysql
import mysql.connector
from sqlalchemy import create_engine
engine = create_engine("mysql+pymysql://" + 'root' + ":" + 'halifax2020' + "@" + "localhost" + "/" + "traice2")

In [31]:
dataset = pd.read_csv('/Users/fabianreginold/Downloads/traice/traice_moneylaundering/PS_20174392719_1491204439457_log.csv')

In [32]:
dataset.to_sql('aml', con = engine, if_exists = 'replace',index = False, chunksize = 1000)

In [33]:
dataset.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0
